# Property Clustring

In [ ]:
# libraries
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## Structured Data Exploration

In [ ]:
#Step 1 — Load only what is needed
use_cols = [
    "PropertyID",
    "Currentavmvalue",
    "equityValue",
    "EquityPercent",
    "LOO",
    "Age",
    "FinalOwnerType",
    "Beds",
    "FullBaths",
    "HalfBaths",
    "Sqft",
    "building_condition",
    "owneroccupied",
    "multi_owner",
    "Lien"
]

df_combined = pd.read_csv(os.path.join("data", "combined_output.csv"),
    usecols=use_cols,
    low_memory=False
)

# print df shape
print("df_combined shape: ", df_combined.shape)

In [ ]:
df_combined.head(5)

### check/handle missing data

In [ ]:
# check for missing values in percentage
for col in df_combined.columns:
    print(col, round(df_combined[col].isna().sum() / len(df_combined) * 100,2), "%")


In [ ]:
# Structurally reliable features (near-complete)
tier1_col = [
   'Currentavmvalue',
   'equityValue',
   'EquityPercent',
   'FinalOwnerType',
   'Age',
   'LOO' 
]

In [ ]:
# Step 2.1 — Encode informative missingness
df_combined["LOO_missing"] = df_combined["LOO"].isna().astype(int)
df_combined["Age_missing"] = df_combined["Age"].isna().astype(int)


In [ ]:
# step 2.2 log transformation of LOO:
# because Early tenure differences matter a lot, but Very long tenure differences matter less, so keeping LOO linear, means: “90 → 100 years is as important as 20 → 30 years.” 
# Which is almost certainly false behaviorally.

df_combined["LOO_log"] = np.log1p(df_combined["LOO"])


In [ ]:
# Step 2.3 - Encode Categorical features
df_combined['is_AO'] = (df_combined['FinalOwnerType'] == 'AO').astype(int)
#df_combined['is_OO'] = df_combined['FinalOwnerType'] == 'OO'  # redundant, as is_AO = 1 - is_OO

In [ ]:
#Step 3.1 — AVM bins (binned current market valuation of the property) --> to be used at sampleing stage not clustering
df_combined["AVM_bin"] = pd.qcut(  # creates Equal-width bins
    df_combined["Currentavmvalue"],
    q=10,
    duplicates="drop"
)

#step 3.2 - log transformation
df_combined["Currentavmvalue_log"] = np.log1p(df_combined["Currentavmvalue"])

#### # Age: Cap at 100 years
##### “Age matters… until it doesn’t.”

stop pretending we can distinguish degrees of ‘very old’ in a meaningful way”

Without capping:
- Age = 300 can dominate distance vs Age = 80
- Even though both are “old” from an investor perspective
- And even though other variables (equity, tenure) already encode the real signal

In [ ]:
# Age: Cap at 100 years
df_combined["is_very_old"] = (df_combined["Age"] > 100).astype(int)  # a secondary binary feature

df_combined["Age_capped"] = df_combined["Age"].clip(upper=100)


In [ ]:
df_combined["Age_capped"].plot.hist(bins=30, edgecolor="black", rwidth=0.9)

In [ ]:
#Step 4 - Data cleaning
# 1.drop rows with EquityPercent >100%
df_combined = df_combined[df_combined["EquityPercent"] <= 100]

# EquityPercent == 100 must be treated as a state (is_fully_paid)
df_combined["is_fully_paid"] = (df_combined["EquityPercent"] == 100).astype(int)

# create continuous part of 'EquityPercent'
df_combined["EquityPercent_cont"] = df_combined["EquityPercent"].where(
    df_combined["EquityPercent"] < 100
)


###  Stratified sampling

In [ ]:
# Step 4 — Stratified sampling
target_size = 150_000
sample_frac = target_size / len(df_combined)

strata_cols = [
    #"FinalOwnerType",
    "AVM_bin",
    "LOO_missing", # to get same fraction of missingness in the sample
    "Age_missing"
]


#--
#- Large strata contribute many rows
#- Small but important strata are preserved
#- Rare combinations don’t disappear
#--
sampled_df = (
    df_combined
    .groupby(strata_cols, group_keys=False, observed=False)
    .apply(lambda x: x.sample(
        frac=sample_frac,
        random_state=42
    ))
)

# step 5 — Save
sampled_df.to_csv(os.path.join("data",
    "combined_output_stratified_sample.csv"),
    index=False
)

NOTE: this sample is a faithful representation of the core investor-targetable population

In [ ]:
sampled_df.shape

In [ ]:
sampled_df["FinalOwnerType"].value_counts(normalize=True)

In [ ]:
df_combined["FinalOwnerType"].value_counts(normalize=True)

In [ ]:
# Is Missingness preserved? (this is critical)
sampled_df[["LOO_missing", "Age_missing"]].mean()


In [ ]:
df_combined[["LOO_missing", "Age_missing"]].mean()

In [ ]:
# AVM distribution sanity
sampled_df["Currentavmvalue"].describe()

In [ ]:
df_combined['Currentavmvalue'].describe()

Note:

the max in the sample vs original df shows discrapency.

Are properties above $1.1M strategically meaningful for investor conversion?

- If no → proceed as-is. The sample is fine. (Going with this option for now)

- If yes → keep top 0.1% by AVM (sample separately) and sample the rest and then combine 

### Investigation

In [ ]:
sampled_df['FinalOwnerType'].value_counts(normalize=True, dropna=False)

==> 'FinalOwnerType' is dangerous and valuable. should not let FinalOwnerType directly drive distance in the first clustering pass. Should be excluded from distance metric to avoid trivial soltion:
“Cluster 1 = AO, Cluster 2 = OO”

In [ ]:
sampled_df.groupby("FinalOwnerType")[
    ["EquityPercent", "LOO_missing"]
].mean()

Conclusion:
- Missing LOO is not explained by ownership type
- AO tend to be further along the financial lifecycle/ But many OO are also high-equity/ And many AO are still leveraged

so 'FinalOwnerType' should inform ranking, not geometry. It carries secondary interpretation.

In [ ]:
# make a 2 by 2 plot
plt.figure(figsize=(30, 20))
# tier1_col exclufing final owner type
col = [c for c in tier1_col if c != "FinalOwnerType"]

for i, col in enumerate(col):
    ax = plt.subplot(3, 2, i + 1)
    ax.set_title(col,fontsize=20)
    sampled_df[col].plot.hist(bins=30, edgecolor="black", rwidth=0.9)

    missing_pct = sampled_df[col].isna().mean() * 100

    ax.text(
        0.98, 0.95,
        f"Missing: {missing_pct:.1f}%",
        transform=ax.transAxes,
        ha="right",
        va="top",
        fontsize=14,
        bbox=dict(boxstyle="round,pad=0.3", alpha=0.3)
    )
    
    plt.tight_layout()

### Observation:

### 1. EquityPercent = 100 is a proxy for ownership psychology, not just finance.

From a behavioral standpoint, these owners are different:

- Often long LOO
- Often older properties
- Frequently absentee owners
- Emotionally detached from the property
- Much easier investor conversations

==> EquityPercent == 100, is_fully_paid? or EquityPercent_capped?

In [ ]:
# How many are fully paid?
(sampled_df['is_fully_paid']).mean()

In [ ]:
# Relationship with LOO
sampled_df.groupby(sampled_df['is_fully_paid'])["LOO"].describe()

In [ ]:
# plot LOO histogram for fully paid

sampled_df[sampled_df['is_fully_paid']==1]["LOO"].plot.hist(bins=30, edgecolor="black", rwidth=0.9)

# set labels
plt.xlabel("LOO (For fully paid properties)")
plt.ylabel("Frequency")

meanLoo = round(sampled_df[sampled_df['is_fully_paid']==1]["LOO"].mean(),1)
# print the mean value
#plt.text(100, 5000, f'mean LOO:{meanLoo}')
plt.text(
        0.3, 0.3,
        f"Mean: {meanLoo:.1f}",
        transform=ax.transAxes,
        ha="right",
        va="top",
        fontsize=14,
        bbox=dict(boxstyle="round,pad=0.3", alpha=0.3))

In [ ]:
sampled_df.head(5)

In [ ]:
# plot AO histograms for fully paid

ownership_pct = sampled_df[sampled_df["is_fully_paid"] == 1]["is_AO"].value_counts(normalize=True) * 100


ax = ownership_pct.rename({1: "Absentee owner", 0: "Owner Occupied"}).plot.bar(
    edgecolor="black",
    width=0.6
)

ax.set_ylabel("Percent of fully paid properties")
ax.set_xlabel("Owner type")
ax.set_title("Owner Type Distribution (Fully Paid Properties)")

# annotate bars
for p in ax.patches:
    height = p.get_height()
    ax.annotate(
        f"{height:.1f}%",
        (p.get_x() + p.get_width() / 2, height),
        ha="center",
        va="bottom"
    )
ax.set_xticklabels(ax.get_xticklabels(), rotation=0)
plt.show()


==> Conclusion: 
- Fully paid ownership and absentee ownership are correlated, but not redundan
- Baseline rate of AO is 52%
- Being fully paid increases the likelihood of being absentee
- is_AO is a good secondary explanatory variable
    - If you had let is_AO into the distance metric --> Clusters would lean heavily toward AO/OO separation

In investor terms, fully paid owners are:
- More likely to be absentee (63.3%)
- More likely to sell directly
- Less constrained by lenders
- But one in three fully paid owners still live in the home (36.7%)

### Missing LOO is not “unknown” — it’s a proxy state

In [ ]:
df_missing_loo = sampled_df[sampled_df["LOO_missing"]==1]
df_non_missing_loo = sampled_df[sampled_df["LOO_missing"]==0]

In [ ]:
def percent_hist(ax, data, bins, xlabel, title):
    weights = np.ones(len(data)) / len(data) * 100

    ax.hist(
        data,
        bins=bins,
        weights=weights,
        edgecolor="black",
        rwidth=0.9
    )

    #ax.set_xlabel(xlabel)
    ax.set_ylabel("Percent of observations")
    ax.set_title(title)
    ax.yaxis.set_major_formatter(lambda y, _: f"{y:.0f}%")

    # annotate bars
    for patch in ax.patches:
        height = patch.get_height()
        if height > 0:
            ax.annotate(
                f"{height:.1f}%",
                (patch.get_x() + patch.get_width() / 2, height),
                ha="center",
                va="bottom",
                fontsize=8
            )

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# --- Row 1: EquityPercent ---
percent_hist(
    ax=axes[0, 0],
    data=df_missing_loo["EquityPercent"],
    bins=20,
    xlabel="EquityPercent",
    title="EquityPercent (LOO missing)"
)

percent_hist(
    ax=axes[0, 1],
    data=sampled_df["EquityPercent"],
    bins=20,
    xlabel="EquityPercent",
    title="EquityPercent (All properties)"
)

# --- Row 2: Age ---
percent_hist(
    ax=axes[1, 0],
    data=sampled_df["Age"].dropna(),
    bins=20,
    xlabel="Age",
    title="Age (All properties)"
)

percent_hist(
    ax=axes[1, 1],
    data=df_missing_loo["Age"].dropna(),
    bins=20,
    xlabel="Age",
    title="Age (LOO missing)"
)

plt.tight_layout()
plt.show()


==> conclusion:
- LOO has two meanings depending on whether it exists.
    - Majority of missing LOO are fully paied properties ~ 71%
- Age distribution does not matter in LOO (Missing LOO is not about the house being older)


In [ ]:
sampled_df[sampled_df['LOO'].notna()]['LOO'].plot.hist(bins=30, edgecolor="black", rwidth=0.9)

In [ ]:
sampled_df['Currentavmvalue'].describe()

## Assembling a model of ownership behavior

#### Step 1 — Sanity check

In [ ]:
cluster_features = [
    "Currentavmvalue_log",
    "is_fully_paid",
    "EquityPercent_cont",
    "LOO_log",
    "LOO_missing",
    "Age_capped"
]

X = sampled_df[cluster_features]

X.info()
X.describe()

#### Step 2 — Handle intentional NaNs

For distance-based clustering, the cleanest option is:

- Replace intentional NaNs with 0
- Rely on the corresponding binary flag to explain why it’s zero

In [ ]:
X_filled = X.fillna(0)

#### Step 3 — Scaling
we have
- Continuous variables (log AVM, LOO_log, Age_capped) ==> comparable in scale
- Binary variables (is_fully_paid, LOO_missing) ==> retain meaning (0 vs 1)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_filled)

In [ ]:
# X_scaled is a numpy array
X_scaled_df = pd.DataFrame(
    X_scaled,
    columns=cluster_features,
    index=sampled_df.index
)

In [ ]:
X_scaled_df.describe()


In [ ]:
plt.figure(figsize=(14, 10))

for i, col in enumerate(cluster_features):
    ax = plt.subplot(3, 2, i + 1)
    X_scaled_df[col].plot.hist(
        bins=30,
        edgecolor="black",
        rwidth=0.9,
        ax=ax
    )
    ax.set_title(col)

plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns

plt.figure(figsize=(8, 6))
sns.heatmap(
    X_scaled_df.corr(),
    annot=True,
    cmap="coolwarm",
    center=0
)
plt.title("Correlation Between Scaled Features")
plt.show()

==> Conclusion: the high correlation is by design, not a problem. e.g.:
- is_fully_paid: Binary state
- EquityPercent_cont: Continuous degree




---

### Algorithm 1: KMeans (first try)

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(
    n_clusters=6,
    random_state=42,
    n_init=20
)

labels = kmeans.fit_predict(X_scaled)

sampled_df["cluster"] = labels

In [ ]:
sampled_df.groupby("cluster")[cluster_features].mean()

In [ ]:
sampled_df.groupby("cluster")["is_AO"].mean()

==> conclusion: Clusters are not trivially “AO vs OO”

In [ ]:
sampled_df.groupby("cluster").size()

### ==> Result interpretation of Algo 1: KMeans (first try)

#### 1. First sanity verdict

Before details:

* ✅ No single cluster dominates (sizes are reasonable)
* ✅ No cluster is tiny/noise
* ✅ Clusters are not trivially “AO vs OO”
* ✅ Equity regime, tenure regime, and AVM are all participating

This means:

> **The geometry is healthy.**

Now we can reason.

---

#### 2. Read the clusters as *ownership personas*

I’ll label them behaviorally (not numerically). These are *working labels*, not final names.

---

##### Cluster 0 — *Legacy Fully Paid, Data-Poor*

* `is_fully_paid = 1`
* `LOO_missing = 1`
* `Age_capped ≈ 66`
* AO ≈ **55%**
* Mid AVM

**Interpretation**
Old ownership, fully paid, missing tenure records. Likely inherited or very long-held properties.

**Investor signal**
Strong. Classic “legacy owner” profile.

---

##### Cluster 2 — *Very Old, Fully Paid, Recorded Tenure*

* `is_fully_paid = 1`
* `LOO_log ≈ 3.01` (very long)
* `Age_capped ≈ 82`
* AO ≈ **58%**
* Lower AVM

**Interpretation**
Physically old homes, very long tenure, fully paid.

**Investor signal**
High motivation potential, but watch condition / rehab risk.

---

##### Cluster 5 — *Fully Paid, High-Value, AO-Dominant*

* `is_fully_paid = 1`
* `LOO_log ≈ 3.00`
* `Age_capped ≈ 43`
* AO ≈ **75%** (highest)
* Highest AVM among fully paid

**Interpretation**
Absentee, fully paid, relatively newer homes, higher value.

**Investor signal**
**Top-tier target cluster**. This is exactly what the business wants.

---

##### Cluster 1 — *Leveraged, Younger, Shorter Tenure*

* `is_fully_paid = 0`
* `EquityPercent ≈ 60`
* `LOO_log ≈ 2.73`
* `Age_capped ≈ 38`
* AO ≈ **45%**
* Highest AVM overall

**Interpretation**
Active ownership, still leveraged, likely listing-oriented.

**Investor signal**
Lower. Probably not your best mailing spend.

---

##### Cluster 3 — *Leveraged, Old Property, Long Tenure*

* `is_fully_paid = 0`
* `EquityPercent ≈ 60`
* `LOO_log ≈ 2.76`
* `Age_capped ≈ 80`
* AO ≈ **38%**

**Interpretation**
Old homes, long tenure, but still leveraged.

**Investor signal**
Mixed. Equity exists, but mortgage friction remains.

---

##### Cluster 4 — *Leveraged, Missing LOO, Mid-Age*

* `is_fully_paid = 0`
* `LOO_missing = 1`
* `EquityPercent ≈ 70`
* `Age_capped ≈ 58`
* AO ≈ **39%**

**Interpretation**
Odd hybrid: higher equity but missing tenure records.

**Investor signal**
Worth investigating. Possibly legacy data + partial refi history.

---

## 3. What this tells us about the model

### ✅ Equity did NOT hijack everything

Fully paid clusters split into **three distinct personas** (0, 2, 5).

That’s exactly what we wanted.

### ✅ LOO_missing is doing real work

Clusters 0 and 4 exist *because* of it.

### ✅ Age is stabilizing, not dominating

Age separates clusters *within* regimes, but doesn’t define them.

### ✅ AO is emergent, not forced

Cluster 5 surfaced as AO-heavy *without* AO in the feature space.

That’s a huge validation.

---

## 4. The most important insight so far

> **Cluster 5 is our gold cluster.**

High AO, fully paid, higher AVM, decent property age.

That is exactly the profile:

* Investors love
* Mail converts
* Margins tend to be higher

Everything we do next should be judged by:

> “Does this sharpen our understanding of Cluster 5 and its neighbors?”

---

## 5. NEXT

### **Step 1 — Stability check **

Run KMeans with different `k` values:

* `k = 5, 6, 7, 8`

Then check:

* Does Cluster 5 persist?
* Does it split meaningfully or dissolve?
* Do legacy clusters remain stable?

If your best cluster vanishes at `k=7`, that’s a warning.

---

### **Step 2 — Translate centroids back to real units**

Right now you’re reading *means*, which is good—but next you should inspect **medians** and **percentiles**, especially for:

* AVM
* EquityPercent
* LOO

Means lie in skewed distributions.

---

### **Step 3 — Attach outcomes (this is where money enters)**

Now we bring in:
`StuckAcqs_4thDecember2025_analysis`

Join on PropertyID (or equivalent), and compute per cluster:

* Conversion rate
* Avg gross margin
* Margin per mailed property

This will answer:

> “Does Cluster 5 actually pay?”

If yes → it becomes your anchor segment.

---

### **Step 4 — Decide if KMeans is “good enough”**

After outcome analysis:

* If clusters are stable and profitable → keep KMeans
* If clusters bleed together → consider:

  * Gaussian Mixture Models (soft membership)
  * Hierarchical clustering (for sub-segmentation)

But **do not jump algorithms yet**.
KMeans has earned its seat so far.

---

## 6. What NOT to do yet

* ❌ Do not optimize silhouette score
* ❌ Do not add AO into features
* ❌ Do not add Sqft/Beds yet
* ❌ Do not over-tune k

We are still in **structure validation**, not optimization.

In [ ]:
# 1. Stability check
from sklearn.cluster import KMeans

k_values = [5, 6, 7, 8]
cluster_results = {}

for k in k_values:
    km = KMeans(
        n_clusters=k,
        random_state=42,
        n_init=20
    )
    labels = km.fit_predict(X_scaled)
    cluster_results[k] = labels

# remove previous clustring column
sampled_df.drop("cluster", axis=1, inplace=True)

for k, labels in cluster_results.items():
    sampled_df[f"cluster_k{k}"] = labels


In [ ]:
# Compare cluster profiles
for k in k_values:
    print(f"\n=== k = {k} ===")
    display(
        sampled_df
        .groupby(f"cluster_k{k}")[cluster_features + ["is_AO"]]
        .mean()
    )

==> Conclusion:
- k = 5 → too coarse
- k = 6 → minimum viable resolution
- k = 7, 8 → refinement, not distortion

In [ ]:
# 2. Compute their membership overlap

gold_k6 = sampled_df.loc[
    (sampled_df["cluster_k6"] == 5)  # 5 is the gold cluster id in k=6
].index

gold_k7 = sampled_df.loc[
    (sampled_df["cluster_k7"] == 0)  # 0 is the gold cluster id in k=7
].index

overlap = len(gold_k6.intersection(gold_k7)) / len(gold_k6)
overlap

==> Conclusion:

- high overlap means: The same properties are being grouped together
- Increasing k is not changing membership
- The model is just refining the rest of the space by increasing k

So, This cluster is not an artifact of k. It is a true basin in the data geometry.

### Profit Validation: Does this gold cluster actually make more money?

In [ ]:
# read csv file
df_stuckAcqs = pd.read_csv(os.path.join("data", "StuckAcqs_4thDecember2025_analysis.csv"))

# print df length
print("df_stuckAcqs shape: ", df_stuckAcqs.shape)

In [ ]:
df_stuckAcqs.columns

In [ ]:
df_stuckAcqs["PropertyID_join"] = (
    df_stuckAcqs["FA Prop Id"]
    .dropna()
    .astype("int64")
    .astype(str)
)

df_combined["PropertyID_join"] = df_combined["PropertyID"].astype(str)
sampled_df["PropertyID_join"] = sampled_df["PropertyID"].astype(str)


In [ ]:
# verify overlap
overlap_rate = df_stuckAcqs["PropertyID_join"].isin(
    sampled_df["PropertyID_join"]
).mean()

overlap_rate

In [ ]:
sampled_joined = sampled_df.merge(
    df_stuckAcqs[["PropertyID_join", "Gross Margin"]],
    on="PropertyID_join",
    how="left"
)

In [ ]:
sampled_joined["Gross Margin"].notna().mean()

In [ ]:
# rename cluster_k6 to cluster
sampled_joined.rename(columns={"cluster_k6": "cluster"}, inplace=True)

In [ ]:
sampled_joined.head()

In [ ]:
sampled_joined.groupby("cluster")["Gross Margin"].apply(
    lambda x: x.notna().mean()
)

==> conclusion: the “gold cluster” is not just interpretable — it actually converts more.

In [ ]:
# Conversion counts by cluster & toral / average margin

sampled_joined.groupby("cluster").agg(
    total_props=("PropertyID_join", "size"),
    converted=("Gross Margin", lambda x: x.notna().sum()),
    avg_margin=("Gross Margin", "mean"),
    total_margin=("Gross Margin", "sum")
)

==> Conclusion:

- Cluster 5
    - $113,582 / 26,935 ≈ $4.22 per property

- Cluster 1
    - $72,390 / 41,735 ≈ $1.73 per property

- Cluster 3
    - $42,178 / 25,647 ≈ $1.64 per property

#### Profile Cluster 5 deeper

- should check the location of the converted properties in this cluster (in the center or at the extreams, ...)
- is the cluster tight enough or should we look at sub-clusters inside it?

In [ ]:
sampled_cluster_5 = sampled_joined[sampled_joined['cluster']==5]

In [ ]:
len(sampled_cluster_5), sampled_cluster_5["Gross Margin"].notna().sum()


In [ ]:
cols_to_check = [
    "Currentavmvalue_log",
    "Age_capped",
    "LOO_log",
    "is_AO"
]

sampled_cluster_5.groupby(sampled_cluster_5["Gross Margin"].notna())[cols_to_check].describe()


==> conclusion:
- Conversions are not edge cases of Cluster 5.
- They sit near the center, with mild enrichment on AVM and Age, and a strong AO signal.
- No need to sub-cluster Cluster 5

### Scale this model to the full dataset

In [ ]:
X_full = df_combined[cluster_features].copy()
X_full_filled = X_full.fillna(0)

In [ ]:
X_full_scaled = scaler.transform(X_full_filled)
X_full_scaled.shape


In [ ]:
df_combined["cluster"] = kmeans.predict(X_full_scaled)


In [ ]:
df_combined["cluster"].value_counts(normalize=True).sort_index()

==> Conclusion:
- Full universe ≈ 5.3M
- Cluster 5 ≈ 17.8%  --> almost exactly macthes the sampled dataset fraction

That’s on the order of ~940k properties, We’re not mailing all of them --> Need to be selective inside Cluster 5

In [ ]:
sampled_df["cluster_k6"].value_counts(normalize=True).sort_index()

In [ ]:
df_combined[df_combined["cluster"] == 5][cluster_features].describe()

In [ ]:
df_combined_joined = df_combined.merge(
    df_stuckAcqs[["PropertyID_join", "Gross Margin"]],
    on="PropertyID_join",
    how="left"
)

df_combined_joined["Gross Margin"].notna().mean()


In [ ]:
df_combined_joined.groupby("cluster").agg(
    total_props=("PropertyID_join", "size"),
    converted=("Gross Margin", lambda x: x.notna().sum()),
    avg_margin=("Gross Margin", "mean"),
    total_margin=("Gross Margin", "sum")
)


In [ ]:
df_combined_joined.groupby("cluster").apply(
    lambda x: x["Gross Margin"].sum() / len(x)
).rename("margin_per_property")


---
### Apply the existing model to df_stuckAcqs


In [ ]:
df_stuckAcqs.head()

In [ ]:
df_stuckAcqs['Equity'] = df_stuckAcqs['Equity'].str.replace('%', '')
df_stuckAcqs["Equity_pct"] = pd.to_numeric(
    df_stuckAcqs["Equity"],
    errors="coerce"
)
df_stuckAcqs['Equity_pct'].head(20)

In [ ]:
df_stuckAcqs["Equity_pct"].isna().mean()

In [ ]:
# Reconstructing the cluster features in the stuckAcqs dataset
cluster_features = [
    "Currentavmvalue_log",
    "is_fully_paid",
    "EquityPercent_cont",
    "LOO_log",
    "LOO_missing",
    "Age_capped"
]

cluster_features

In [ ]:
import numpy as np

# 1) AVM (log)
df_stuckAcqs["Currentavmvalue_log"] = np.log1p(df_stuckAcqs["AVM"])

# 2) is_fully_paid (strict, conservative definition)
df_stuckAcqs["is_fully_paid"] = (
    (df_stuckAcqs["Equity_pct"] == 100) &
    (df_stuckAcqs["Total Mortgage"].fillna(0) <= 1)
).astype(int)

# 3) EquityPercent_cont (only when not fully paid)
df_stuckAcqs["EquityPercent_cont"] = df_stuckAcqs["Equity_pct"].where(
    df_stuckAcqs["Equity_pct"] < 100
)

# 4) LOO_log
df_stuckAcqs["LOO_log"] = np.log1p(df_stuckAcqs["LOO"])

# 5) LOO_missing
df_stuckAcqs["LOO_missing"] = df_stuckAcqs["LOO"].isna().astype(int)

# 6) Age_capped
df_stuckAcqs["Age_capped"] = df_stuckAcqs["Age"].clip(upper=100)


In [ ]:
# Missingness check
df_stuckAcqs[
    [
        "Currentavmvalue_log",
        "is_fully_paid",
        "EquityPercent_cont",
        "LOO_log",
        "LOO_missing",
        "Age_capped"
    ]
].isna().mean()

# Sanity distributions
#df_stuckAcqs["is_fully_paid"].value_counts(dropna=False)
#df_stuckAcqs["EquityPercent_cont"].describe()


In [ ]:
# Missingness check
sampled_df[
    [
        "Currentavmvalue_log",
        "is_fully_paid",
        "EquityPercent_cont",
        "LOO_log",
        "LOO_missing",
        "Age_capped"
    ]
].isna().mean()

### ==> Why we cannot apply the model directly on the df_stuckAcqs:

- `df_stuckAcqs` is an outcome / acquisition table, not a feature-complete representation of the mailing universe the clustering model was trained on.
- Several core clustering features are missing at high rates in df_stuckAcqs:
    - ~22% missing AVM
    - ~76% missing equity depth
    - ~53% missing length of ownership (LOO)
- Applying the model directly would break the feature geometry and cause cluster assignments to be driven by missingness and imputed values rather than real property behavior.
- Filtering to “compatible” rows would introduce selection bias, because it conditions on data availability rather than underlying economics or owner behavior.
- Any clusters produced directly on df_stuckAcqs would therefore be methodologically invalid and misleading.
- The correct way to evaluate cluster performance is to join acquisitions onto the full mailing universe and measure expected value per property, which is the approach we used to identify Cluster 5 as the gold segment.

---

---
### Designing a scoring/ ranking model
Who is most likely to convert?

In [ ]:
df5 = df_combined_joined[df_combined_joined["cluster"] == 5]

df5.groupby(df5["Gross Margin"].notna()).agg(
    count=("PropertyID_join", "size"),
    ao_rate=("is_AO", "mean"),
    avg_avm=("Currentavmvalue_log", "mean"),
    avg_age=("Age_capped", "mean"),
    avg_LOO=("LOO_log", "mean"),
    loo_missing_rate=("LOO_missing", "mean")
)

==> Conclusions:

1️. Cluster membership is the base score

- Dominant effect
- Largest lift
- Stable and interpretable

2. Within Cluster 5, refinements are:

- AO → strong positive
- AVM → moderate positive
- Age → optional / very weak (can be ignored initially)

3. LOO does not matter inside Cluster 5

- Consistent with sample
- We explicitly exclude it from the score refinement